In [ ]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest,f_classif

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def create_base_train_data(x,y):
    selector=SelectKBest(score_func=f_classif,k=5).fit(x,y)
    x_select=selector.transform(x)
    select_features=selector.get_support(1)
    new_x,new_y=SMOTE(sampling_strategy=0.2,random_state=SEED).fit_resample(x_select,y)
    new_x=PolynomialFeatures(degree=2).fit_transform(new_x)
    return new_x,new_y,select_features

def create_base_test_data(x,feature):
    new_x=x[x.columns[feature]]
    new_x=PolynomialFeatures(degree=2).fit_transform(new_x)
    return new_x

def predict_score(model,x_train,y_train,x_test,y_test,name):
    model.fit(x_train,y_train)
    y_predict=model.predict(x_test)
    f1=f1_score(y_test,y_predict)
    auc_score=roc_auc_score(y_test,y_predict)
    return f1,auc_score
    
def baseDataSet(LongMethodFile):
    df=pd.read_csv(LongMethodFile,encoding_errors="replace")
    df=df[['fanin_x', 'fanout_x', 'wmc_x', 'loc_x', 'returnsQty_x', 'variablesQty_x', 'parametersQty_x', 'loopQty_x', 'anonymousClassesQty_x', 'innerClassesQty_x','LongMethod']]
    df=df.dropna(how="any")
    return df

def addMetricsDataSet(LongMethodFile):
    df=pd.read_csv(LongMethodFile,encoding_errors="replace")
    df=df[['fanin_x', 'fanout_x', 'wmc_x', 'loc_x', 'returnsQty_x', 'variablesQty_x', 'parametersQty_x', 'loopQty_x', 'anonymousClassesQty_x', 'innerClassesQty_x', 'ATFD_x', 'ATLD_x', 'LAA_x', 'FDP_x',"LongMethod"]]
    df=df.dropna(how="any")
    return df

def addOldVersionDataSet(LongMethodFile):
    df=pd.read_csv(LongMethodFile,encoding_errors="replace")
    df=df[['fanin_x', 'fanout_x', 'wmc_x', 'loc_x', 'returnsQty_x', 'variablesQty_x', 'parametersQty_x', 'loopQty_x', 'anonymousClassesQty_x', 'innerClassesQty_x', 'ATFD_x', 'ATLD_x', 'LAA_x', 'FDP_x', 'fanin_y', 'fanout_y', 'wmc_y', 'loc_y', 'returnsQty_y', 'variablesQty_y', 'parametersQty_y', 'loopQty_y', 'anonymousClassesQty_y', 'innerClassesQty_y', 'ATFD_y', 'ATLD_y', 'LAA_y', 'FDP_y', 'LongMethod']]
    df=df.dropna(how="any")
    return df

def normalization(df):
    for column in df.columns:
        df[column]=(df[column]-df[column].min())/(df[column].max()-df[column].min())
    return df

#オーバーサンプリング，多項式化データ
def create_train_test_data(x,y):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=SEED)
    x_train_base,y_train_base,features=create_base_train_data(x_train,y_train)
    x_test_base=create_base_test_data(x_test,features)
    return x_train_base,y_train_base,x_test_base,y_test,features

SEED=1
cpu=6
datasetFolder=os.path.dirname(os.path.dirname(os.getcwd()))

In [ ]:
LongMethodFile=os.path.join(datasetFolder,"dataset2\\LongMethod.csv")
baseDF=baseDataSet(LongMethodFile)
x_base=baseDF.drop(columns=["LongMethod"])
y_base=baseDF[["LongMethod"]].values.ravel()
x_base=normalization(x_base)
x_train_base,y_train_base,x_test_base,y_test_base,features_base=create_train_test_data(x_base,y_base)

addMetricsDF=addMetricsDataSet(LongMethodFile)
x_add=addMetricsDF.drop(columns=["LongMethod"])
y_add=addMetricsDF[["LongMethod"]].values.ravel()
x_add=normalization(x_add)
x_train_add,y_train_add,x_test_add,y_test_add,features_add=create_train_test_data(x_add,y_add)

c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:


name="KNN"
KNN_prameter={
    'n_neighbors': range(1, 21, 2),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
    }
KNN=KNeighborsClassifier()

KNN_model=RandomizedSearchCV(KNN, param_distributions=KNN_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED,n_iter=cpu)


f1_base,auc_score_base=predict_score(KNN_model,x_train_base,y_train_base,x_test_base,y_test_base,name)
f1_add,auc_score_add=predict_score(KNN_model,x_train_add,y_train_add,x_test_add,y_test_add,name)

print("base")
print("f1 : ",f1_base)
print("auc_score : ",auc_score_base)
print(baseDF.columns[features_base])
print("\naddMetrics")
print("f1 : ",f1_add)
print("auc_score : ",auc_score_add)
print(addMetricsDF.columns[features_add])

c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\

base
f1 :  0.49795918367346936
auc_score :  0.8065675447729925
Index(['wmc_x', 'loc_x', 'variablesQty_x', 'loopQty_x',
       'anonymousClassesQty_x'],
      dtype='object')

addMetrics
f1 :  0.5
auc_score :  0.8265737308998837
Index(['wmc_x', 'loc_x', 'variablesQty_x', 'ATFD_x', 'FDP_x'], dtype='object')


In [2]:
import os
print(os.path.dirname(os.path.dirname(os.getcwd())))

c:\Users\syuuj\machinelearning


feture envy K近傍
base
f1 :  0.14337349397590363
auc_score :  0.5763375827931042
Index(['fanout_x', 'wmc_x', 'loc_x', 'variablesQty_x', 'loopQty_x'], dtype='object')

addMetrics
f1 :  0.25669099756691
auc_score :  0.6420965261185135
Index(['loc_x', 'variablesQty_x', 'loopQty_x', 'ATFD_x', 'FDP_x'], dtype='object')